In [1]:
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
from scipy import stats

In [2]:
def get_even_num(num=63):
    runs = []
    evens = []
    j = 0
    for i in range(num):
        if (i != 0) & (i != 2):
            if i % 2 == 0:
                evens.append(str(i))
                j += 1
                if j == 5: 
                    runs.append(evens)
                    evens = []
                    j = 0
    return runs

In [11]:
data_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/"
save_path = data_path+"raw/"
sub_path_32 = data_path+"external/R21_32_aqueous_2mA/"
sub_path_33 = data_path+"external/R21_33_aqueous_2mA/"
sub_path_34 = data_path+"external/R21_34_aqueous_p2mA/"
sub_path_35 = data_path+"external/R21_35_Cyl_Aqueous/"
sub_path_36 = data_path+"external/R21_36_Cyl_Aqueous/"
sub_path_37 = data_path+"external/R21_37_Cyl_Aqueous/"
sub_path_38 = data_path+"external/R21_38_Cyl_Aqueous/"
#cwd_path = sub_path+"/NIFTI/bmaps_numpy/"
tes = [2.46, 4.61, 7.09, 10.5, 15.0, 20.0, 24.5, 29.0, 33.5, 38.0, 42.5, 47.0]

# 36 to 38
base_runs = get_even_num()
bp = [['6','4'],['10','4'],['6','10']],[['16','14'],['20','14'],['16','20']],[['26','24'],['30','24'],['26','30']],[['36','34'],['40','34'],['36','40']],[['46','44'],['50','44'],['46','50']],[['56','54'],['60','54'],['56','60']]


In [12]:
base_runs

[['4', '6', '8', '10', '12'],
 ['14', '16', '18', '20', '22'],
 ['24', '26', '28', '30', '32'],
 ['34', '36', '38', '40', '42'],
 ['44', '46', '48', '50', '52'],
 ['54', '56', '58', '60', '62']]

In [13]:
bp

([['6', '4'], ['10', '4'], ['6', '10']],
 [['16', '14'], ['20', '14'], ['16', '20']],
 [['26', '24'], ['30', '24'], ['26', '30']],
 [['36', '34'], ['40', '34'], ['36', '40']],
 [['46', '44'], ['50', '44'], ['46', '50']],
 [['56', '54'], ['60', '54'], ['56', '60']])

# Extract Data

## 1. fmap mean of 5 runs

In [15]:
def extract_mean(sub_path, experiment):
    print(sub_path)
    cwd_path = sub_path+"/NIFTI/bmaps_numpy/"
    result = pd.DataFrame()
    for m in range(len(base_runs)):   
        tdir = cwd_path+bp[m][0][0]+"minus"+bp[m][0][1]+"/"
        theory_img = nib.load(tdir+"Bvals.nii")
        theory_data = theory_img.get_data()
        sp = theory_data.shape
        b_data = []

        for i in range(sp[0]):
            for j in range(sp[1]):
                for k in range(sp[2]):
                    b_data.append([experiment, m+1, i, j, k, theory_data[i][j][k]])

        b_df = pd.DataFrame(b_data, columns=['exp', 'mini_exp', 'i', 'j', 'k', 'theory'])

        df = pd.DataFrame()
        for n in range(len(base_runs[m])):
            bdir = cwd_path+"base_run_"+base_runs[m][n]+"/"
            print(base_runs[m][n])

            new_data = []      
            fmap_img = nib.load(bdir+"4d_fmap_Mean.nii.gz")
            fmap_data = fmap_img.get_data()
            sp = fmap_data.shape            
            col = "mean"+str(n)

            for i in range(sp[0]):
                for j in range(sp[1]):
                    for k in range(sp[2]):
                        new_data.append([experiment, m+1, i, j, k, fmap_data[i][j][k]])

            new_df = pd.DataFrame(new_data, columns=['exp', 'mini_exp', 'i', 'j', 'k', col])
            if n == 0:
                df = new_df
            else:
                df = df.merge(new_df, on=['exp', 'mini_exp', 'i', 'j', 'k'], sort=False)


        df = df.merge(b_df, on=['exp','mini_exp', 'i', 'j', 'k'], sort=False)
        result = result.append(df, ignore_index=True)
    return result

In [16]:
# 32 and 33
base_runs = [['4', '6', '8', '10', '12'], ['14', '16', '18', '20', '22']]
bp = [['6','4'],['10','4'],['6','10']],[['16','14'],['20','14'],['16','20']]

In [17]:
result_32 = extract_mean(sub_path_32, '32')
result_33 = extract_mean(sub_path_33, '33')

/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_32_aqueous_2mA/
4
6
8
10
12
14
16
18
20
22
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_33_aqueous_2mA/
4
6
8
10
12
14
16
18
20
22


In [31]:
# 34
base_runs = [['4', '6', '8', '10', '12']]
bp = [['6','4'],['10','4'],['6','10']],

In [32]:
result_34 = extract_mean(sub_path_34, '34')

/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_34_aqueous_p2mA/
4
6
8
10
12


In [33]:
# 35
base_runs = [['4', '6', '8', '10', '12'],
 ['14', '16', '18', '20', '22'],
 ['24', '26', '28', '30', '32'],
 ['34', '36', '38', '40', '42'],
 ['44', '46', '48', '50', '52']]
bp = [['6','4'],['10','4'],['6','10']],[['16','14'],['20','14'],['16','20']],[['26','24'],['30','24'],['26','30']],[['36','34'],['40','34'],['36','40']],[['46','44'],['50','44'],['46','50']]

In [34]:
result_35 = extract_mean(sub_path_35, '35')

/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_35_Cyl_Aqueous/
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52


In [35]:
# 36 to 38
base_runs = get_even_num()
bp = [['6','4'],['10','4'],['6','10']],[['16','14'],['20','14'],['16','20']],[['26','24'],['30','24'],['26','30']],[['36','34'],['40','34'],['36','40']],[['46','44'],['50','44'],['46','50']],[['56','54'],['60','54'],['56','60']]

In [36]:
result_36 = extract_mean(sub_path_36, '36')
result_37 = extract_mean(sub_path_37, '37')
result_38 = extract_mean(sub_path_38, '38')

/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_36_Cyl_Aqueous/
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_37_Cyl_Aqueous/
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_38_Cyl_Aqueous/
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62


In [37]:
print(result_32.shape)
print(result_33.shape)
print(result_34.shape)
print(result_35.shape)
print(result_36.shape)
print(result_37.shape)
print(result_38.shape)

(360448, 11)
(360448, 11)
(180224, 11)
(901120, 11)
(1081344, 11)
(1081344, 11)
(1081344, 11)


In [38]:
result = result_36.append(result_37, ignore_index=True)
result = result.append(result_38, ignore_index=True)
result = result.append(result_32, ignore_index=True)
result = result.append(result_33, ignore_index=True)
result = result.append(result_34, ignore_index=True)
# result = result.append(result_35, ignore_index=True)
result

,exp,mini_exp,i,j,k,mean0,mean1,mean2,mean3,mean4,theory
0,36,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,36,1,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0
2,36,1,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0
3,36,1,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0
4,36,1,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0
5,36,1,0,0,5,0.0,0.0,0.0,0.0,0.0,0.0
6,36,1,0,0,6,0.0,0.0,0.0,0.0,0.0,0.0
7,36,1,0,0,7,0.0,0.0,0.0,0.0,0.0,0.0
8,36,1,0,0,8,0.0,0.0,0.0,0.0,0.0,0.0
9,36,1,0,0,9,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
result.shape

(4145152, 11)

In [41]:
result_32[(result_32['i']==31) & (result_32['j']==31) & (result_32['k']==31)]

,exp,mini_exp,i,j,k,mean0,mean1,mean2,mean3,mean4,theory
128991,32,1,31,31,31,557.953796,581.584106,534.349670,497.107605,514.994507,43.305984
309215,32,2,31,31,31,571.918640,613.776978,577.935974,542.774048,581.564697,43.305984


In [42]:
np.savetxt(save_path+"fmap_mean_32to38.txt", result.values, fmt="%s")

## 2. 55 attributes

In [32]:
def extract_mean_55(sub_path, experiment):
    print(sub_path)
    cwd_path = sub_path+"/NIFTI/bmaps_numpy/"
    result = pd.DataFrame()
    for m in range(len(base_runs)):   
        tdir = cwd_path+bp[m][0][0]+"minus"+bp[m][0][1]+"/"
        theory_img = nib.load(tdir+"Bvals.nii")
        theory_data = theory_img.get_data()
        sp = theory_data.shape
        b_data = []

        for i in range(sp[0]):
            for j in range(sp[1]):
                for k in range(sp[2]):
                    b_data.append([experiment, m+1, i, j, k, theory_data[i][j][k]])

        b_df = pd.DataFrame(b_data, columns=['exp', 'mini_exp', 'i', 'j', 'k', 'theory'])

        df = pd.DataFrame()
        for n in range(len(base_runs[m])):
            bdir = cwd_path+"base_run_"+base_runs[m][n]+"/"
            print(base_runs[m][n])
                
            temp_df = pd.DataFrame()
            for l in range(1, 12):
                new_data = []
                fname = "fmap"+str(l+1)+"m"+str(l)
                fmap_img = nib.load(bdir+fname+".nii.gz")
                fmap_data = fmap_img.get_data()
                sp = fmap_data.shape            
                col = base_runs[0][n]+"_"+fname
                
                for i in range(sp[0]):
                    for j in range(sp[1]):
                        for k in range(sp[2]):
                            new_data.append([experiment, m+1, i, j, k, fmap_data[i][j][k]])
        
                new_df = pd.DataFrame(new_data, columns=['exp', 'mini_exp', 'i', 'j', 'k', col])
                if l == 1:
                    temp_df = new_df
                else:
                    temp_df = temp_df.merge(new_df, on=['exp', 'mini_exp', 'i', 'j', 'k'], sort=False)
            
            if n == 0:
                df = temp_df
            else:
                df = df.merge(temp_df, on=['exp', 'mini_exp', 'i', 'j', 'k'], sort=False)

        df = df.merge(b_df, on=['exp','mini_exp', 'i', 'j', 'k'], sort=False)
        result = result.append(df, ignore_index=True)
    return result

In [33]:
result_55_36 = extract_mean_55(sub_path_36, '36')
result_55_37 = extract_mean_55(sub_path_37, '37')
result_55_38 = extract_mean_55(sub_path_38, '38')

/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_36_Cyl_Aqueous/
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_37_Cyl_Aqueous/
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_38_Cyl_Aqueous/
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62


In [34]:
print(result_55_36.shape)
print(result_55_37.shape)
print(result_55_38.shape)

(1081344, 61)
(1081344, 61)
(1081344, 61)


In [35]:
result_55 = result_55_36.append(result_55_37, ignore_index=True)
result_55 = result_55.append(result_55_38, ignore_index=True)
result_55

,exp,mini_exp,i,j,k,4_fmap2m1,4_fmap3m2,4_fmap4m3,4_fmap5m4,4_fmap6m5,...,12_fmap4m3,12_fmap5m4,12_fmap6m5,12_fmap7m6,12_fmap8m7,12_fmap9m8,12_fmap10m9,12_fmap11m10,12_fmap12m11,theory
0,36,1,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,36,1,0,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,36,1,0,0,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,36,1,0,0,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,36,1,0,0,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,36,1,0,0,5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,36,1,0,0,6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,36,1,0,0,7,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,36,1,0,0,8,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,36,1,0,0,9,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
result_55.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3244032 entries, 0 to 3244031
Data columns (total 61 columns):
exp             object
mini_exp        int64
i               int64
j               int64
k               int64
4_fmap2m1       float64
4_fmap3m2       float64
4_fmap4m3       float64
4_fmap5m4       float64
4_fmap6m5       float64
4_fmap7m6       float64
4_fmap8m7       float64
4_fmap9m8       float64
4_fmap10m9      float64
4_fmap11m10     float64
4_fmap12m11     float64
6_fmap2m1       float64
6_fmap3m2       float64
6_fmap4m3       float64
6_fmap5m4       float64
6_fmap6m5       float64
6_fmap7m6       float64
6_fmap8m7       float64
6_fmap9m8       float64
6_fmap10m9      float64
6_fmap11m10     float64
6_fmap12m11     float64
8_fmap2m1       float64
8_fmap3m2       float64
8_fmap4m3       float64
8_fmap5m4       float64
8_fmap6m5       float64
8_fmap7m6       float64
8_fmap8m7       float64
8_fmap9m8       float64
8_fmap10m9      float64
8_fmap11m10     float64
8_fmap12

In [37]:
result_55[(result_55['i']==31) & (result_55['j']==31) & (result_55['k']==31)]

,exp,mini_exp,i,j,k,4_fmap2m1,4_fmap3m2,4_fmap4m3,4_fmap5m4,4_fmap6m5,...,12_fmap4m3,12_fmap5m4,12_fmap6m5,12_fmap7m6,12_fmap8m7,12_fmap9m8,12_fmap10m9,12_fmap11m10,12_fmap12m11,theory
128991,36,1,31,31,31,436.606018,406.205139,424.674500,437.552673,432.719299,...,345.783661,358.691345,353.730835,339.612030,325.620575,312.901093,314.172943,317.988861,324.348633,34.932812
309215,36,2,31,31,31,346.090210,318.501831,337.385162,351.063995,345.717499,...,273.600922,290.010284,282.755646,272.198456,253.119049,244.215439,242.943481,244.215439,251.847198,34.932812
489439,36,3,31,31,31,279.534149,249.262421,270.243835,284.922455,278.176575,...,221.566452,232.767899,228.955872,216.232407,197.153214,188.249298,185.705566,188.249390,194.609192,34.932812
669663,36,4,31,31,31,223.627487,193.870605,214.852295,228.951813,223.232056,...,171.210266,186.977524,182.016769,169.174484,150.090683,139.915207,138.643051,142.459122,148.818726,34.932812
849887,36,5,31,31,31,175.707138,147.711121,167.853439,183.161453,176.293121,...,132.604111,147.546768,143.094971,129.739532,113.208244,105.572289,103.028366,103.028366,108.116203,34.932812
1030111,36,6,31,31,31,138.436172,110.783257,129.247025,143.731094,139.660675,...,100.712120,111.932083,108.752045,95.396805,80.133087,69.957603,68.685646,69.961617,75.045441,34.932812
1210335,37,1,31,31,31,724.127319,697.018311,710.019653,722.470642,717.764465,...,691.555969,703.391296,701.737793,693.215759,676.680237,666.504700,657.600952,660.149109,661.416748,32.912647
1390559,37,2,31,31,31,708.153442,678.554382,691.555969,704.663269,699.448242,...,679.805908,690.671814,685.711182,679.224182,663.960754,653.785095,647.425293,647.429504,649.969177,32.912647
1570783,37,3,31,31,31,692.180420,667.015137,681.484497,693.215759,688.000671,...,668.056396,680.496155,676.553040,667.776611,652.513123,643.609375,633.433899,633.437927,635.977661,32.912647
1751007,37,4,31,31,31,681.531433,653.167175,671.413513,679.224060,676.553162,...,657.985168,669.048584,666.250244,658.872925,641.065552,632.161743,621.986206,624.534607,627.073914,32.912647


In [38]:
np.savetxt(save_path+"fmap_mean_55.txt", result_55.values, fmt="%s")

## 3. Separate +ON and -ON

In [202]:
def extract_on(sub_path, experiment):
    print(sub_path)
    cwd_path = sub_path+"/NIFTI/bmaps_numpy/"

    temp_df = pd.DataFrame()
    result = pd.DataFrame()
    for m in range(len(base_runs)):   
        run1 = 1
        run2 = -1
        tdir1 = cwd_path+bp[m][0][0]+"minus"+bp[m][0][1]+"/"
        theory1_img = nib.load(tdir1+"Bvals.nii")
        theory1_data = theory1_img.get_data()
        tdir2 = cwd_path+bp[m][1][0]+"minus"+bp[m][1][1]+"/"
        theory2_img = nib.load(tdir2+"Bvals.nii")
        theory2_data = theory2_img.get_data()
        
        sp = theory1_data.shape
        b1_data = []
        b2_data = []

        for i in range(sp[0]):
            for j in range(sp[1]):
                for k in range(sp[2]):
                    b1_data.append([experiment, m+1, run1, i, j, k, theory1_data[i][j][k]])
                    b2_data.append([experiment, m+1, run2, i, j, k, theory2_data[i][j][k]])

        b_df = pd.DataFrame(b1_data, columns=['exp', 'mini_exp', 'run', 'i', 'j', 'k', 'theory'])
        b2_df = pd.DataFrame(b2_data, columns=['exp', 'mini_exp', 'run', 'i', 'j', 'k', 'theory'])
        b_df = b_df.append(b2_df, ignore_index=True)
        
        df = pd.DataFrame()
        for n in range(len(base_runs[m])):
            if n==2 or n==4:
                continue
            
            bdir = cwd_path+"base_run_"+base_runs[m][n]+"/"

            new_data = []   
            fmap_img = nib.load(bdir+"4d_fmap_Mean.nii.gz")
            fmap_data = fmap_img.get_data()
            sp = fmap_data.shape            
            col = "on"
            
            if n==0:
                fmap0_data = fmap_img.get_data()
                col0 = "off"
                continue
            elif n==1:
                run = run1
            elif n==3:
                run = run2

            for i in range(sp[0]):
                for j in range(sp[1]):
                    for k in range(sp[2]):
                        new_data.append([experiment, m+1, run, i, j, k, fmap0_data[i][j][k], fmap_data[i][j][k]])

            new_df = pd.DataFrame(new_data, columns=['exp', 'mini_exp', 'run', 'i', 'j', 'k', col0, col])
            if n == 1:
                temp_df = new_df
            else:
                temp_df = temp_df.append(new_df)

        if m == 0:
            df = temp_df
        else:
            df = df.append(temp_df)

        df = df.merge(b_df, on=['exp','mini_exp', 'run', 'i', 'j', 'k'], sort=False)
        result = result.append(df, ignore_index=True)
    return result

In [203]:
result_on_36 = extract_on(sub_path_36, '36')
result_on_37 = extract_on(sub_path_37, '37')
result_on_38 = extract_on(sub_path_38, '38')

/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_36_Cyl_Aqueous/
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_37_Cyl_Aqueous/
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_38_Cyl_Aqueous/


In [204]:
print(result_on_36.shape)
print(result_on_37.shape)
print(result_on_38.shape)

(2162688, 9)
(2162688, 9)
(2162688, 9)


In [205]:
result_on = result_on_36.append(result_on_37, ignore_index=True)
result_on = result_on.append(result_on_38, ignore_index=True)
result_on

,exp,mini_exp,run,i,j,k,off,on,theory
0,36,1,1,0,0,0,0.0,0.0,0.0
1,36,1,1,0,0,1,0.0,0.0,0.0
2,36,1,1,0,0,2,0.0,0.0,0.0
3,36,1,1,0,0,3,0.0,0.0,0.0
4,36,1,1,0,0,4,0.0,0.0,0.0
5,36,1,1,0,0,5,0.0,0.0,0.0
6,36,1,1,0,0,6,0.0,0.0,0.0
7,36,1,1,0,0,7,0.0,0.0,0.0
8,36,1,1,0,0,8,0.0,0.0,0.0
9,36,1,1,0,0,9,0.0,0.0,0.0


In [206]:
result_on_36[(result_on_36['i']==22) & (result_on_36['j']==50) & (result_on_36['k']==31)]

,exp,mini_exp,run,i,j,k,off,on,theory
93343,36,1,1,22,50,31,981.519409,1037.164062,60.290371
273567,36,1,-1,22,50,31,981.519409,827.297791,-60.290371
453791,36,2,1,22,50,31,889.942810,952.236267,51.133694
634015,36,2,-1,22,50,31,889.942810,752.791931,-51.133694
814239,36,3,1,22,50,31,821.282715,887.185547,51.133694
994463,36,3,-1,22,50,31,821.282715,695.186707,-51.133694
1174687,36,4,1,22,50,31,764.547974,829.719055,51.133694
1354911,36,4,-1,22,50,31,764.547974,640.817139,-51.133694
1535135,36,5,1,22,50,31,721.863708,791.985535,51.133694
1715359,36,5,-1,22,50,31,721.863708,603.434021,-51.133694


In [208]:
np.savetxt(save_path+"fmap_on.txt", result_on.values, fmt="%s")

## 4. Physics Model

In [14]:
def extract_mean_correction(sub_path, experiment):
    print(sub_path)
    cwd_path = sub_path+"/NIFTI/bmaps_numpy/"
    result = pd.DataFrame()
    for m in range(len(base_runs)):   
        tdir = cwd_path+bp[m][0][0]+"minus"+bp[m][0][1]+"/"
        theory_img = nib.load(tdir+"4d_fmap_Mean.nii.gz")
        theory_data = theory_img.get_data()
        sp = theory_data.shape
        b_data = []

        for i in range(sp[0]):
            for j in range(sp[1]):
                for k in range(sp[2]):
                    b_data.append([experiment, m+1, i, j, k, theory_data[i][j][k]])

        b_df = pd.DataFrame(b_data, columns=['exp', 'mini_exp', 'i', 'j', 'k', 'mean'])
        
        result = result.append(b_df, ignore_index=True)
    return result

In [15]:
result_correct_36 = extract_mean_correction(sub_path_36, '36')
result_correct_37 = extract_mean_correction(sub_path_37, '37')
result_correct_38 = extract_mean_correction(sub_path_38, '38')

/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_36_Cyl_Aqueous/
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_37_Cyl_Aqueous/
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_38_Cyl_Aqueous/


In [16]:
print(result_correct_36.shape)
print(result_correct_37.shape)
print(result_correct_38.shape)

(1081344, 6)
(1081344, 6)
(1081344, 6)


In [17]:
result_correct = result_correct_36.append(result_correct_37, ignore_index=True)
result_correct = result_correct.append(result_correct_38, ignore_index=True)
result_correct

,exp,mini_exp,i,j,k,mean
0,36,1,0,0,0,0.0
1,36,1,0,0,1,0.0
2,36,1,0,0,2,0.0
3,36,1,0,0,3,0.0
4,36,1,0,0,4,0.0
5,36,1,0,0,5,0.0
6,36,1,0,0,6,0.0
7,36,1,0,0,7,0.0
8,36,1,0,0,8,0.0
9,36,1,0,0,9,0.0


In [18]:
result_correct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3244032 entries, 0 to 3244031
Data columns (total 6 columns):
exp         object
mini_exp    int64
i           int64
j           int64
k           int64
mean        float64
dtypes: float64(1), int64(4), object(1)
memory usage: 148.5+ MB


In [19]:
result_correct[(result_correct['i']==31) & (result_correct['j']==31) & (result_correct['k']==31)]

,exp,mini_exp,i,j,k,mean
128991,36,1,31,31,31,13.247223
309215,36,2,31,31,31,18.652618
489439,36,3,31,31,31,21.574554
669663,36,4,31,31,31,22.524246
849887,36,5,31,31,31,24.645630
1030111,36,6,31,31,31,26.587303
1210335,37,1,31,31,31,25.956970
1390559,37,2,31,31,31,28.541077
1570783,37,3,31,31,31,28.199646
1751007,37,4,31,31,31,27.852722


In [22]:
np.savetxt(save_path+"fmap_mean_physics.txt", result_correct.values, fmt="%s")

# Play Ground

In [28]:
def extract_correction(cwd_path, experiment):
    result = pd.DataFrame()
    
    theory_img = nib.load(cwd_path+"vol0001.nii.gz")
    theory_data = theory_img.get_data()
    sp = theory_data.shape
    b_data = []

    for i in range(sp[0]):
        for j in range(sp[1]):
            for k in range(sp[2]):
                b_data.append([experiment, i, j, k, theory_data[i][j][k]])

    b_df = pd.DataFrame(b_data, columns=['exp', 'i', 'j', 'k', 'value'])

    result = result.append(b_df, ignore_index=True)
    return result

In [29]:
path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/"
result = extract_correction(path, '36')

In [30]:
result

,exp,i,j,k,value
0,36,0,0,0,0.0
1,36,0,0,1,0.0
2,36,0,0,2,0.0
3,36,0,0,3,0.0
4,36,0,0,4,0.0
5,36,0,0,5,0.0
6,36,0,0,6,0.0
7,36,0,0,7,0.0
8,36,0,0,8,0.0
9,36,0,0,9,0.0


In [31]:
np.savetxt(path+"correction.txt", result.values, fmt="%s")